In [1]:
from pyspark import SparkContext
import pyspark

conf = pyspark.SparkConf().set("spark.driver.host", "localhost")
sc = SparkContext.getOrCreate()
sc.setLogLevel('OFF')

21/11/19 11:33:11 WARN Utils: Your hostname, Fans-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.1.143.47 instead (on interface en0)
21/11/19 11:33:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/19 11:33:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/11/19 11:33:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/11/19 11:33:13 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/11/19 11:33:13 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
supervisor_sf_pair = sc.textFile("../data/SF_business/supervisor_sf.tsv")\
                       .map(lambda x : x.split("\t"))\
                       .map(lambda x : (x[0], x[1]))

In [3]:
zip_biz_pair = sc.textFile("../data/SF_business/filtered_registered_business_sf.tsv")\
                 .map(lambda x : x.split("\t"))\
                 .map(lambda x : (x[0], x[1:]))

## 1. Return any five distinct  zip code in zip_biz_pair

In [4]:
zip_biz_pair.take(5)

[('94123', ['Tournahu George L', '3301 Broderick St', 'San Francisco', 'CA']),
 ('94124',
  ['Stephens Institute Inc', '2225 Jerrold Ave', 'San Francisco', 'CA']),
 ('94105',
  ['Stephens Institute Inc', '180 New Montgomery St', 'San Francisco', 'CA']),
 ('94108', ['Stephens Institute Inc', '540 Powell St', 'San Francisco', 'CA']),
 ('94107',
  ['Stephens Institute Inc', '460 Townsend St', 'San Francisco', 'CA'])]

In [6]:
zip_biz_pair.keys().take(5)

['94123', '94124', '94105', '94108', '94107']

## 2. Return any five distinct city names.

In [10]:
zip_biz_pair.values().map(lambda x: x[2]).distinct().take(5)

['San Francisco', 'Hayward', 'Redwood+city', 'Daly+city', 'Mexico']

## 3. Return five distinct (zip code, city name) pairs ordered by zip code, where zip code has 5 digits.

In [21]:
distinct_zip_city_pairs = zip_biz_pair.filter(lambda x: len(x[0])==5).mapValues(lambda x: x[2]).sortByKey().distinct()

In [22]:
distinct_zip_city_pairs.take(5)

[('10001', 'New+york'),
 ('10004', 'New+york'),
 ('10005', 'New York'),
 ('10006', 'New York'),
 ('10007', 'New+york')]

## 4. Find the zip associated with the most city names
* which one is more efficient ??
    - zip_associated_with_multiple_cities_1  $vs$ zip_associated_with_multiple_cities_2

In [ ]:
zip_biz_pair.groupByKey()

### Which one is more efficient? v1 or v2??

#### 

## 5. Create zip_city_count_names which includes (zip, (count, the list of names))

In [ ]:
zip_city_count_names = 

In [ ]:
zip_city_count_names.take(1)

## 6. convert zip_city_count_names to (zip, city_name)

In [ ]:
zip_city_count_names.

In [ ]:
sc.stop()